#In this notebook we load our fine-tuned BERT basedmodels and evaluate them on the validation and test sets

# Install

In [ ]:
!pip install -U -q datasets transformers torchinfo

     |████████████████████████████████| 441 kB 19.4 MB/s 
     |████████████████████████████████| 5.3 MB 40.3 MB/s 
     |████████████████████████████████| 163 kB 73.2 MB/s 
     |████████████████████████████████| 115 kB 54.7 MB/s 
     |████████████████████████████████| 212 kB 31.8 MB/s 
     |████████████████████████████████| 127 kB 70.6 MB/s 
     |████████████████████████████████| 7.6 MB 55.4 MB/s 
     |████████████████████████████████| 115 kB 74.4 MB/s 


# Imports

In [ ]:
from datasets import Dataset, DatasetDict, ClassLabel, Value, load_dataset, load_metric
from huggingface_hub import notebook_login
from torchinfo import summary
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import pandas as pd
from sklearn.metrics import classification_report

# Data loading

In [ ]:
# login to HF
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
# load dataset
ds = load_dataset('sara-nabhani/lfd-proj',
    'csv', 
    data_files={'train': 'train.csv', 'val': 'val.csv', 'test': 'test.csv'}
)

cl = ClassLabel(names=list(ds['train'].unique('label')))
ds = ds.cast_column('label', cl)

ds

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 12240
    })
    val: Dataset({
        features: ['tweet', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 860
    })
})

# bert-base-cased-finetuned-0 Model Evaluation

In [ ]:
model_id = 'sara-nabhani/bert-base-cased-fintuned-0'
#load tokenizer and tokenize dataset
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-0/snapshots/e81c7ea00e68bb25cc60943bf7674a8c2d0a9f27/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-0/snapshots/e81c7ea00e68bb25cc60943bf7674a8c2d0a9f27/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-0/snapshots/e81c7ea00e68bb25cc60943bf7674a8c2d0a9f27/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-0/snapshots/e81c7ea00e68bb25cc60943bf7674a8c2d0a9f27/tokenizer_config.json


In [ ]:
metric = load_metric("accuracy")
metric1= load_metric("f1")
# evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"acc": metric.compute(predictions=predictions, references=labels), "f1":metric1.compute(predictions=predictions, references=labels, average='macro')}
#load model
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
trainer = Trainer(
  model=model,
  train_dataset=tokenized_ds['train'],
  eval_dataset=tokenized_ds['val'],
  compute_metrics=compute_metrics,
  tokenizer=tokenizer,
  data_collator=data_collator
  )

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-0/snapshots/e81c7ea00e68bb25cc60943bf7674a8c2d0a9f27/config.json
Model config BertConfig {
  "_name_or_path": "sara-nabhani/bert-base-cased-fintuned-0",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "OFF",
    "1": "NOT"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NOT": 1,
    "OFF": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  

## Evaluation

In [ ]:
print("Prediction on validation data:")
print(trainer.predict(tokenized_ds['val'])[-1])

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Prediction on validation data:


{'test_loss': 0.4839080274105072, 'test_acc': {'accuracy': 0.79}, 'test_f1': {'f1': 0.7686164586418447}, 'test_runtime': 35.2114, 'test_samples_per_second': 28.4, 'test_steps_per_second': 3.55}


In [ ]:
print("Prediction on test data:")
print(trainer.predict(tokenized_ds['test'])[-1])

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


Prediction on test data:


{'test_loss': 0.3805697560310364, 'test_acc': {'accuracy': 0.8511627906976744}, 'test_f1': {'f1': 0.8073840769903763}, 'test_runtime': 30.2254, 'test_samples_per_second': 28.453, 'test_steps_per_second': 3.573}


## Saving to DF

In [ ]:
val_predictions = np.argmax(trainer.predict(tokenized_ds['val'])[0], axis=-1)
test_predictions = np.argmax(trainer.predict(tokenized_ds['test'])[0], axis=-1)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


In [ ]:
val_data_frame=pd.DataFrame({'tweet': ds['val']['tweet'], 'true_label': ds['val']['label'], 'pred_label': val_predictions})
test_data_frame=pd.DataFrame({'tweet': ds['test']['tweet'], 'true_label': ds['test']['label'], 'pred_label': test_predictions})

In [ ]:
print("Validation classification report:")
print(classification_report(val_data_frame['true_label'], val_data_frame['pred_label']))

Validation classification report:
              precision    recall  f1-score   support

           0       0.71      0.69      0.70       352
           1       0.83      0.84      0.84       648

    accuracy                           0.79      1000
   macro avg       0.77      0.77      0.77      1000
weighted avg       0.79      0.79      0.79      1000



In [ ]:
print("Test classification report:")
print(classification_report(test_data_frame['true_label'], test_data_frame['pred_label']))

Test classification report:
              precision    recall  f1-score   support

           0       0.77      0.67      0.72       240
           1       0.88      0.92      0.90       620

    accuracy                           0.85       860
   macro avg       0.82      0.80      0.81       860
weighted avg       0.85      0.85      0.85       860



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
test_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/models-output/bert-base-cased-finetuned-0.csv', index=False)
val_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/val-models-output/bert-base-cased-finetuned-0.csv', index=False)

# bert-base-cased-finetuned-1 Model Evaluation

In [ ]:
model_id = 'sara-nabhani/bert-base-cased-fintuned-1'
#load tokenizer and tokenize dataset
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)

print(len(tokenized_ds['train']['input_ids'][0]))
metric = load_metric("accuracy")
metric1= load_metric("f1")
# evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"acc": metric.compute(predictions=predictions, references=labels), "f1":metric1.compute(predictions=predictions, references=labels, average='macro')}
#load model
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
trainer = Trainer(
  model=model,
  train_dataset=tokenized_ds['train'],
  eval_dataset=tokenized_ds['val'],
  compute_metrics=compute_metrics,
  tokenizer=tokenizer,
  data_collator=data_collator
  )
#evaluation
print("Prediction on validation data:")
print(trainer.predict(tokenized_ds['val'])[-1])
print("Prediction on test data:")
print(trainer.predict(tokenized_ds['test'])[-1])
#saving to DF
val_predictions = np.argmax(trainer.predict(tokenized_ds['val'])[0], axis=-1)
test_predictions = np.argmax(trainer.predict(tokenized_ds['test'])[0], axis=-1)
val_data_frame=pd.DataFrame({'tweet': ds['val']['tweet'], 'true_label': ds['val']['label'], 'pred_label': val_predictions})
test_data_frame=pd.DataFrame({'tweet': ds['test']['tweet'], 'true_label': ds['test']['label'], 'pred_label': test_predictions})
print("Validation classification report:")
print(classification_report(val_data_frame['true_label'], val_data_frame['pred_label']))
print("Test classification report:")
print(classification_report(test_data_frame['true_label'], test_data_frame['pred_label']))
test_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/models-output/bert-base-cased-fintuned-1.csv', index=False)
val_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/val-models-output/bert-base-cased-fintuned-1.csv', index=False)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-1/snapshots/9e714e379af2718b93532be9eef430816f41affb/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-1/snapshots/9e714e379af2718b93532be9eef430816f41affb/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-1/snapshots/9e714e379af2718b93532be9eef430816f41affb/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-1/snapshots/9e714e379af2718b93532be9eef430816f41affb/tokenizer_config.json


512


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-1/snapshots/9e714e379af2718b93532be9eef430816f41affb/config.json
Model config BertConfig {
  "_name_or_path": "sara-nabhani/bert-base-cased-fintuned-1",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "OFF",
    "1": "NOT"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NOT": 1,
    "OFF": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  

Prediction on validation data:


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


{'test_loss': 0.4948499798774719, 'test_acc': {'accuracy': 0.795}, 'test_f1': {'f1': 0.7733602058110444}, 'test_runtime': 35.0764, 'test_samples_per_second': 28.509, 'test_steps_per_second': 3.564}
Prediction on test data:


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


{'test_loss': 0.37620607018470764, 'test_acc': {'accuracy': 0.8383720930232558}, 'test_f1': {'f1': 0.790530228026614}, 'test_runtime': 29.9926, 'test_samples_per_second': 28.674, 'test_steps_per_second': 3.601}


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


Validation classification report:
              precision    recall  f1-score   support

           0       0.72      0.69      0.70       352
           1       0.84      0.85      0.84       648

    accuracy                           0.80      1000
   macro avg       0.78      0.77      0.77      1000
weighted avg       0.79      0.80      0.79      1000

Test classification report:
              precision    recall  f1-score   support

           0       0.74      0.65      0.69       240
           1       0.87      0.91      0.89       620

    accuracy                           0.84       860
   macro avg       0.81      0.78      0.79       860
weighted avg       0.83      0.84      0.83       860



# bert-base-cased-finetuned-2 Model Evaluation

In [ ]:
model_id = 'sara-nabhani/bert-base-cased-fintuned-2'
#load tokenizer and tokenize dataset
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True, max_length=100)

tokenized_ds = ds.map(tokenize_function, batched=True)

print(len(tokenized_ds['train']['input_ids'][0]))
metric = load_metric("accuracy")
metric1= load_metric("f1")
# evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"acc": metric.compute(predictions=predictions, references=labels), "f1":metric1.compute(predictions=predictions, references=labels, average='macro')}
#load model
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
trainer = Trainer(
  model=model,
  train_dataset=tokenized_ds['train'],
  eval_dataset=tokenized_ds['val'],
  compute_metrics=compute_metrics,
  tokenizer=tokenizer,
  data_collator=data_collator
  )
#evaluation
print("Prediction on validation data:")
print(trainer.predict(tokenized_ds['val'])[-1])
print("Prediction on test data:")
print(trainer.predict(tokenized_ds['test'])[-1])
#saving to DF
val_predictions = np.argmax(trainer.predict(tokenized_ds['val'])[0], axis=-1)
test_predictions = np.argmax(trainer.predict(tokenized_ds['test'])[0], axis=-1)
val_data_frame=pd.DataFrame({'tweet': ds['val']['tweet'], 'true_label': ds['val']['label'], 'pred_label': val_predictions})
test_data_frame=pd.DataFrame({'tweet': ds['test']['tweet'], 'true_label': ds['test']['label'], 'pred_label': test_predictions})
print("Validation classification report:")
print(classification_report(val_data_frame['true_label'], val_data_frame['pred_label']))
print("Test classification report:")
print(classification_report(test_data_frame['true_label'], test_data_frame['pred_label']))
test_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/models-output/bert-base-cased-fintuned-2.csv', index=False)
val_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/val-models-output/bert-base-cased-fintuned-2.csv', index=False)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-2/snapshots/cffafc001441d16b2d3bb34ecd64cfbc3ed168c5/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-2/snapshots/cffafc001441d16b2d3bb34ecd64cfbc3ed168c5/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-2/snapshots/cffafc001441d16b2d3bb34ecd64cfbc3ed168c5/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-2/snapshots/cffafc001441d16b2d3bb34ecd64cfbc3ed168c5/tokenizer_config.json


100


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-2/snapshots/cffafc001441d16b2d3bb34ecd64cfbc3ed168c5/config.json
Model config BertConfig {
  "_name_or_path": "sara-nabhani/bert-base-cased-fintuned-2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "OFF",
    "1": "NOT"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NOT": 1,
    "OFF": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  

Prediction on validation data:


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


{'test_loss': 0.4887142479419708, 'test_acc': {'accuracy': 0.794}, 'test_f1': {'f1': 0.7730237641915239}, 'test_runtime': 6.3006, 'test_samples_per_second': 158.716, 'test_steps_per_second': 19.839}
Prediction on test data:


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


{'test_loss': 0.37155917286872864, 'test_acc': {'accuracy': 0.8395348837209302}, 'test_f1': {'f1': 0.7911313542319309}, 'test_runtime': 5.4414, 'test_samples_per_second': 158.049, 'test_steps_per_second': 19.848}


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


Validation classification report:
              precision    recall  f1-score   support

           0       0.71      0.70      0.70       352
           1       0.84      0.85      0.84       648

    accuracy                           0.79      1000
   macro avg       0.77      0.77      0.77      1000
weighted avg       0.79      0.79      0.79      1000

Test classification report:
              precision    recall  f1-score   support

           0       0.75      0.64      0.69       240
           1       0.87      0.92      0.89       620

    accuracy                           0.84       860
   macro avg       0.81      0.78      0.79       860
weighted avg       0.83      0.84      0.84       860



# bert-base-cased-finetuned-3 Model Evaluation

In [ ]:
model_id = 'sara-nabhani/bert-base-cased-fintuned-3'
#load tokenizer and tokenize dataset
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True, max_length=200)

tokenized_ds = ds.map(tokenize_function, batched=True)

print(len(tokenized_ds['train']['input_ids'][0]))
metric = load_metric("accuracy")
metric1= load_metric("f1")
# evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"acc": metric.compute(predictions=predictions, references=labels), "f1":metric1.compute(predictions=predictions, references=labels, average='macro')}
#load model
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
trainer = Trainer(
  model=model,
  train_dataset=tokenized_ds['train'],
  eval_dataset=tokenized_ds['val'],
  compute_metrics=compute_metrics,
  tokenizer=tokenizer,
  data_collator=data_collator
  )
#evaluation
print("Prediction on validation data:")
print(trainer.predict(tokenized_ds['val'])[-1])
print("Prediction on test data:")
print(trainer.predict(tokenized_ds['test'])[-1])
#saving to DF
val_predictions = np.argmax(trainer.predict(tokenized_ds['val'])[0], axis=-1)
test_predictions = np.argmax(trainer.predict(tokenized_ds['test'])[0], axis=-1)
val_data_frame=pd.DataFrame({'tweet': ds['val']['tweet'], 'true_label': ds['val']['label'], 'pred_label': val_predictions})
test_data_frame=pd.DataFrame({'tweet': ds['test']['tweet'], 'true_label': ds['test']['label'], 'pred_label': test_predictions})
print("Validation classification report:")
print(classification_report(val_data_frame['true_label'], val_data_frame['pred_label']))
print("Test classification report:")
print(classification_report(test_data_frame['true_label'], test_data_frame['pred_label']))
test_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/models-output/bert-base-cased-fintuned-3.csv', index=False)
val_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/val-models-output/bert-base-cased-fintuned-3.csv', index=False)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-3/snapshots/834db08c182de1e2364035e67ee568649ad5f7c9/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-3/snapshots/834db08c182de1e2364035e67ee568649ad5f7c9/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-3/snapshots/834db08c182de1e2364035e67ee568649ad5f7c9/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-3/snapshots/834db08c182de1e2364035e67ee568649ad5f7c9/tokenizer_config.json


200


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--bert-base-cased-fintuned-3/snapshots/834db08c182de1e2364035e67ee568649ad5f7c9/config.json
Model config BertConfig {
  "_name_or_path": "sara-nabhani/bert-base-cased-fintuned-3",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "OFF",
    "1": "NOT"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NOT": 1,
    "OFF": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  

Prediction on validation data:


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


{'test_loss': 0.49006927013397217, 'test_acc': {'accuracy': 0.788}, 'test_f1': {'f1': 0.765781053896873}, 'test_runtime': 12.7905, 'test_samples_per_second': 78.183, 'test_steps_per_second': 9.773}
Prediction on test data:


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


{'test_loss': 0.3795793652534485, 'test_acc': {'accuracy': 0.8395348837209302}, 'test_f1': {'f1': 0.7899007923862935}, 'test_runtime': 11.0566, 'test_samples_per_second': 77.782, 'test_steps_per_second': 9.768}


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


Validation classification report:
              precision    recall  f1-score   support

           0       0.71      0.68      0.69       352
           1       0.83      0.85      0.84       648

    accuracy                           0.79      1000
   macro avg       0.77      0.76      0.77      1000
weighted avg       0.79      0.79      0.79      1000

Test classification report:
              precision    recall  f1-score   support

           0       0.75      0.63      0.69       240
           1       0.87      0.92      0.89       620

    accuracy                           0.84       860
   macro avg       0.81      0.78      0.79       860
weighted avg       0.83      0.84      0.84       860

